In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train
from manual_nn import nn_pass
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
data, params = prd.load_caps_train('traindata.mat')

In [20]:
trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max = prd.prep_train_caps(data, params)

In [3]:
ep = 30
n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler, x_min, x_max = prd.prep_train_caps(data, params)

# Train neural networks
models = [mlp, mlp_beta, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    train_mod = get_train()

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y in ds:
            train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train aligned LDA
y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

mlp_enc = mlp.get_layer(name='enc')
w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

mlpbeta_enc = mlp_beta.get_layer(name='enc')
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

cnn_enc = cnn.get_layer(name='enc')
temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn.get_weights()

Epoch 1, Loss: 1.88, Accuracy: 24.55 
Epoch 30, Loss: 0.55, Accuracy: 79.33 
Epoch 1, Loss: 1.07, Accuracy: 62.46 
Epoch 30, Loss: 0.13, Accuracy: 95.28 
Epoch 1, Loss: 0.92, Accuracy: 69.97 
Epoch 30, Loss: 0.11, Accuracy: 96.03 


In [27]:
w_in = mlpb_w

In [61]:
import csv

In [82]:
# Neural network architectures
arch = ['RELU1','BN1','RELU2','BN2','RELU3','BN3','DEN','BN4','SOFTMAX']
w = {}

i = 0
for l in arch:
    w_layer = []
    if 'BN' in l:
        w_layer = np.vstack((np.vstack((np.vstack((w_in[i],w_in[i+1])),w_in[i+2])),w_in[i+3]))
        i += 4
    else:
        w_layer = np.vstack((w_in[i],w_in[i+1]))
        i += 2
    with open('weights/' + l + '.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(w_layer)
    w[l] = w_layer

with open('weights/ARCH.csv', 'w', newline='') as f:
        writer = csv.writer(f,delimiter=',')
        # write multiple rows
        for i in arch:
            writer.writerow([i])

fill = len(emg_scale) - len(x_min)
temp = np.hstack((np.vstack((x_min[...,np.newaxis],np.zeros((fill,1)))), np.vstack((x_max[...,np.newaxis],np.zeros((fill,1))))))
scales = np.hstack((emg_scale,temp))

with open('weights/scales.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        # write multiple rows
        writer.writerows(scales)

with open('weights.p','wb') as f:
    pickle.dump(emg_scale,f,protocol=2, fix_imports=True)
    # pickle.dump([w,arch,emg_scale,x_min,x_max],f, protocol=2)

In [58]:
test = np.ones((1,2)).astype('byte')
test = 1.2
# with open('weights.p','rb') as f:
#     emg_scale = pickle.load(f)
with open('weights.p','wb') as f:
    pickle.dump(test,f,protocol=2, fix_imports=True)

In [42]:
temp_max = -1*np.ones((1,2))
for i in range(len(mlpb_w)):
    temp_size = mlpb_w[i].shape
    if mlpb_w[i].ndim < 2:
        temp_size = temp_size[...,np.newaxis]
    temp_max = np.vstack((temp_max,temp_size)).max(axis=0)


TypeError: tuple indices must be integers or slices, not tuple

In [35]:
fill = len(emg_scale) - len(x_min)
temp = np.hstack((np.vstack((x_min[...,np.newaxis],np.zeros((fill,1)))), np.vstack((x_max[...,np.newaxis],np.zeros((fill,1))))))
test = np.hstack((emg_scale,temp))

In [39]:
for i in range(len(mlpb_w)):
    print(mlpb_w[i].shape)

(24, 246)
(246,)
(246,)
(246,)
(246,)
(246,)
(246, 128)
(128,)
(128,)
(128,)
(128,)
(128,)
(128, 16)
(16,)
(16,)
(16,)
(16,)
(16,)
(16, 4)
(4,)
(4,)
(4,)
(4,)
(4,)
(4, 7)
(7,)


In [36]:
import csv

with open('mlpb_w.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write multiple rows
    writer.writerows(mlpb_w)

with open('scales.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write multiple rows
    writer.writerows(test)